In [1]:
import json
import os
import sys

import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

from koopmanlib.dictionary import PsiNN_obs
from koopmanlib.param_solver import (
    KoopmanBilinearDLSolver,
    KoopmanLinearDLSolver,
    KoopmanParametricDLSolver,
)

from tqdm.keras import TqdmCallback

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# config_file = sys.argv[1]
config_file = '/home/guoyue/Learning-Parametric-Koopman-Decompositions/examples/ParametricKoopman/kdv/config_kdv.json'

with open(config_file) as f:
    config = json.load(f)["sin"]

data_path = config["data_settings"]["data_path"]
weights_path = config["nn_settings"]["weights_path"]
forcing_type = config["data_settings"]["forcing_type"]

n_psi_train = config["nn_settings"]["n_psi_train"]


Nx = config["data_settings"]["Nx"]
L = 2 * np.pi
dx = L / (Nx - 1.0)
target_dim = Nx
param_dim = 3

n_psi = 1 + 1 + 1 + n_psi_train

dict_layer_size = config["nn_settings"]["dict_layer_size"]
K_layer_size = config["nn_settings"]["K_layer_size"]

linear_epochs = config["nn_settings"]["linear_epochs"]
bilinear_epochs = config["nn_settings"]["bilinear_epochs"]
pknn_epochs = config["nn_settings"]["pknn_epochs"]


print('K_layer_size', K_layer_size)


2024-07-28 20:23:28.223410: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-28 20:23:29.306461: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-28 20:23:33.996473: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.10.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.1/lib64:
2024-07-28 20:23:33.997644: W tensorflow/compiler/xla/stre

K_layer_size [36, 36]


In [3]:


# Load data
dict_data = np.load(
    os.path.join(data_path, "data_kdv_" + forcing_type + ".npy"), allow_pickle=True
)

data_x = dict_data[()]["data_x"]
data_y = dict_data[()]["data_y"]
data_u = dict_data[()]["data_u"]

# PK-NN
dic_pk = PsiNN_obs(layer_sizes=dict_layer_size, n_psi_train=n_psi_train, dx=dx)
from koopmanlib.K_structure import Model_K_u_Layer_One

model_K_u = Model_K_u_Layer_One(layer_sizes=K_layer_size, n_psi=n_psi, activation="relu")

solver_pk = KoopmanParametricDLSolver(
    target_dim=target_dim, param_dim=param_dim, n_psi=n_psi, dic=dic_pk, model_K_u=model_K_u
)

model_pk, model_K_u_pred_pk = solver_pk.generate_model()

model_pk.summary()

zeros_data_y_train = tf.zeros_like(dic_pk(data_y))

model_pk.compile(optimizer=Adam(0.001), loss="mse")


2024-07-28 20:42:04.633459: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-28 20:42:04.633497: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: neon
2024-07-28 20:42:04.633503: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: neon
2024-07-28 20:42:04.633608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.171.4
2024-07-28 20:42:04.633632: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.171.4
2024-07-28 20:42:04.633637: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 535.171.4
2024-07-28 20:42:04.633926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf.math.square (TFOpLambda)    (None, 128)          0           ['input_3[0][0]']                
                                                                                                  
 input_4 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf.math.reduce_sum (TFOpLambda  (None,)             0           ['input_3[0][0]']                
 )                                                                                            

In [ ]:

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="loss",
    factor=0.1,
    patience=50,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=1e-12,
)
checkpoint_path = os.path.join(weights_path, "pk_kdv_weights_" + forcing_type + ".h5")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True,
        mode="min",
        save_freq="epoch",
    )

# Define the TqdmCallback for progress bar
tqdm_callback = TqdmCallback(verbose=1)

callbacks = [lr_callback, checkpoint_callback, tqdm_callback]

history_pk = model_pk.fit(
    x=[data_x, data_y, data_u],
    y=zeros_data_y_train,
    validation_split=0.2,
        epochs=pknn_epochs,
        batch_size=200,
        callbacks=callbacks,
        verbose=0
    )

training_loss = history_pk.history['loss']
validation_loss = history_pk.history['val_loss']
best_epoch = validation_loss.index(min(validation_loss))
best_loss_pk = training_loss[best_epoch]
best_val_loss_pk = validation_loss[best_epoch]


# Linear Model: Dynamics is $Az +Bu$

dic_linear = PsiNN_obs(layer_sizes=dict_layer_size, n_psi_train=n_psi_train, dx=dx)

solver_linear = KoopmanLinearDLSolver(
    dic=dic_linear, target_dim=target_dim, param_dim=param_dim, n_psi=n_psi
)

model_linear, model_K_u_pred_linear = solver_linear.build_model()

solver_linear.build(
    model_linear,
    data_x,
    data_u,
    data_y,
    zeros_data_y_train,
    epochs=linear_epochs,
    batch_size=200,
    lr=0.0001,
    lr_patience=100,
    lr_decay_factor=0.1,
    lr_min=1e-10,
    es_patience=50,
    es_min_delta=1e-9,
    filepath=os.path.join(weights_path, "linear_kdv_weights_" + forcing_type + ".h5"))

best_loss_linear = solver_linear.loss_best_model
best_val_loss_linear = solver_linear.val_loss_best_model


# Bilinear Model: Dynamics is $Az + \sum_{i=1}^{N_{u}}B_{i}zu_{i}$

dic_bilinear = PsiNN_obs(layer_sizes=dict_layer_size, n_psi_train=n_psi_train, dx=dx)

solver_bilinear = KoopmanBilinearDLSolver(
    dic=dic_bilinear, target_dim=target_dim, param_dim=param_dim, n_psi=n_psi
)

model_bilinear, model_K_u_pred_bilinear = solver_bilinear.build_model()

solver_bilinear.build(
    model_bilinear,
    data_x,
    data_u,
    data_y,
    zeros_data_y_train,
    epochs=linear_epochs,
    batch_size=200,
    lr=0.0001,
    lr_patience=100,
    lr_decay_factor=0.1,
    lr_min=1e-10,
    es_patience=50,
    es_min_delta=1e-9,
    filepath=os.path.join(weights_path, "bilinear_kdv_weights_" + forcing_type + ".h5")
)

best_loss_bilinear = solver_bilinear.loss_best_model
best_val_loss_bilinear = solver_bilinear.val_loss_best_model


loss_dict = {'loss_pk': best_loss_pk,
             'val_loss_pk': best_val_loss_pk, 
             'loss_linear': best_loss_linear, 
             'val_loss_linear': best_val_loss_linear, 
             'loss_bilinear': best_loss_bilinear, 
             'val_loss_bilinear': best_val_loss_bilinear}

import pandas as pd

# Convert the dictionary to a DataFrame
df = pd.DataFrame([loss_dict])

# Save the DataFrame to a CSV file
df.to_csv('loss_values_kdv'+str(forcing_type)+'.csv', index=False)

## Add a vanilla NN model

In [4]:
data_x.shape

(200000, 128)

In [5]:
data_u.shape

(200000, 3)

In [6]:
data_y.shape

(200000, 128)

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Define the input layers
input_x = layers.Input(shape=(128,))
input_u = layers.Input(shape=(3,))

# Concatenate the inputs
concatenated = layers.Concatenate()([input_x, input_u])

# Hidden layer
hidden_layer = layers.Dense(20, activation='relu')(concatenated) 

# Output layer
output_y = layers.Dense(128, activation='linear')(hidden_layer)

# Create the model
model_nonlinear = models.Model(inputs=[input_x, input_u], outputs=output_y)

# Print model summary to check the number of parameters
model_nonlinear.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 128)]        0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, 3)]          0           []                               
                                                                                                  
 concatenate_5 (Concatenate)    (None, 131)          0           ['input_10[0][0]',               
                                                                  'input_11[0][0]']               
                                                                                                  
 dense_10 (Dense)               (None, 20)           2640        ['concatenate_5[0][0]']    

__________________________________________________________________________________________________


In [17]:
# Compile the model
model_nonlinear.compile(optimizer='adam', loss='mean_squared_error')

In [18]:
history_nonlinear = model_nonlinear.fit([data_x, data_u], data_y, epochs=pknn_epochs, batch_size=200)

Epoch 1/500
1000/1000 [==============================] - 2s 1ms/step - loss: 0.0042
Epoch 2/500
1000/1000 [==============================] - 1s 1ms/step - loss: 2.4046e-04
Epoch 3/500
1000/1000 [==============================] - 1s 1ms/step - loss: 1.3097e-04
Epoch 4/500
1000/1000 [==============================] - 1s 1ms/step - loss: 7.3358e-05
Epoch 5/500
1000/1000 [==============================] - 1s 1ms/step - loss: 3.9952e-05
Epoch 6/500
1000/1000 [==============================] - 1s 1ms/step - loss: 2.5072e-05
Epoch 7/500
1000/1000 [==============================] - 1s 1ms/step - loss: 1.9338e-05
Epoch 8/500
1000/1000 [==============================] - 1s 1ms/step - loss: 1.5895e-05
Epoch 9/500
1000/1000 [==============================] - 1s 1ms/step - loss: 1.4036e-05
Epoch 10/500
1000/1000 [==============================] - 1s 1ms/step - loss: 1.3210e-05
Epoch 11/500
1000/1000 [==============================] - 1s 1ms/step - loss: 1.2676e-05
Epoch 12/500
1000/1000 [==========

In [20]:
vanilla_nn_weights_path = os.path.join(weights_path, "vanilla_nn_kdv_weights_" + forcing_type + ".h5")

In [21]:
model_nonlinear.save_weights(vanilla_nn_weights_path)